### Look at Portland 2024 CVRs for city council STV election and get them into a nicer format.
### This notebook: District 4

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd

pd.set_option('display.max_columns', None)

import numpy as np

import maup
import warnings; warnings.filterwarnings('ignore', 'GeoSeries.isna', UserWarning)
maup.progress.enabled = True

from tqdm import tqdm

In [2]:
D4_df = pd.read_csv("./Data/Cast Vote Records/City_of_Portland__Councilor__District_4_2024_11_29_17_26_12.cvr.csv")

In [3]:
D4_df

,RowNumber,BoxID,BoxPosition,BallotID,PrecinctID,BallotStyleID,PrecinctStyleName,ScanComputerName,Status,Remade,"Choice_88_1:City of Portland, Councilor, District 4:1:Number of Winners 3:Bob Weinstein:NON","Choice_88_1:City of Portland, Councilor, District 4:2:Number of Winners 3:Bob Weinstein:NON","Choice_88_1:City of Portland, Councilor, District 4:3:Number of Winners 3:Bob Weinstein:NON","Choice_88_1:City of Portland, Councilor, District 4:4:Number of Winners 3:Bob Weinstein:NON","Choice_88_1:City of Portland, Councilor, District 4:5:Number of Winners 3:Bob Weinstein:NON","Choice_88_1:City of Portland, Councilor, District 4:6:Number of Winners 3:Bob Weinstein:NON","Choice_89_1:City of Portland, Councilor, District 4:1:Number of Winners 3:Andra Vltavin:NON","Choice_89_1:City of Portland, Councilor, District 4:2:Number of Winners 3:Andra Vltavin:NON","Choice_89_1:City of Portland, Councilor, District 4:3:Number of Winners 3:Andra Vltavin:NON","Choice_89_1:City of Portland, Councilor, District 4:4:Number of Winners 3:Andra Vltavin:NON","Choice_89_1:City of Portland, Councilor, District 4:5:Number of Winners 3:Andra Vltavin:NON","Choice_89_1:City of Portland, Councilor, District 4:6:Number of Winners 3:Andra Vltavin:NON","Choice_90_1:City of Portland, Councilor, District 4:1:Number of Winners 3:Lee Odell:NON","Choice_90_1:City of Portland, Councilor, District 4:2:Number of Winners 3:Lee Odell:NON","Choice_90_1:City of Portland, Councilor, District 4:3:Number of Winners 3:Lee Odell:NON","Choice_90_1:City of Portland, Councilor, District 4:4:Number of Winners 3:Lee Odell:NON","Choice_90_1:City of Portland, Councilor, District 4:5:Number of Winners 3:Lee Odell:NON","Choice_90_1:City of Portland, Councilor, District 4:6:Number of Winners 3:Lee Odell:NON","Choice_91_1:City of Portland, Councilor, District 4:1:Number of Winners 3:Kevin Goldsmith:NON","Choice_91_1:City of Portland, Councilor, District 4:2:Number of Winners 3:Kevin Goldsmith:NON","Choice_91_1:City of Portland, Councilor, District 4:3:Number of Winners 3:Kevin Goldsmith:NON","Choice_91_1:City of Portland, Councilor, District 4:4:Number of Winners 3:Kevin Goldsmith:NON","Choice_91_1:City of Portland, Councilor, District 4:5:Number of Winners 3:Kevin Goldsmith:NON","Choice_91_1:City of Portland, Councilor, District 4:6:Number of Winners 3:Kevin Goldsmith:NON","Choice_92_1:City of Portland, Councilor, District 4:1:Number of Winners 3:John J Goldsmith:NON","Choice_92_1:City of Portland, Councilor, District 4:2:Number of Winners 3:John J Goldsmith:NON","Choice_92_1:City of Portland, Councilor, District 4:3:Number of Winners 3:John J Goldsmith:NON","Choice_92_1:City of Portland, Councilor, District 4:4:Number of Winners 3:John J Goldsmith:NON","Choice_92_1:City of Portland, Councilor, District 4:5:Number of Winners 3:John J Goldsmith:NON","Choice_92_1:City of Portland, Councilor, District 4:6:Number of Winners 3:John J Goldsmith:NON","Choice_93_1:City of Portland, Councilor, District 4:1:Number of Winners 3:Mitch Green:NON","Choice_93_1:City of Portland, Councilor, District 4:2:Number of Winners 3:Mitch Green:NON","Choice_93_1:City of Portland, Councilor, District 4:3:Number of Winners 3:Mitch Green:NON","Choice_93_1:City of Portland, Councilor, District 4:4:Number of Winners 3:Mitch Green:NON","Choice_93_1:City of Portland, Councilor, District 4:5:Number of Winners 3:Mitch Green:NON","Choice_93_1:City of Portland, Councilor, District 4:6:Number of Winners 3:Mitch Green:NON","Choice_94_1:City of Portland, Councilor, District 4:1:Number of Winners 3:Joseph (Joe) Alfone:NON","Choice_94_1:City of Portland, Councilor, District 4:2:Number of Winners 3:Joseph (Joe) Alfone:NON","Choice_94_1:City of Portland, Councilor, District 4:3:Number of Winners 3:Joseph (Joe) Alfone:NON","Choice_94_1:City of Portland, Councilor, District 4:4:Number of Winners 3:Joseph (Joe) Alfone:NON","Choice_94_1:City of Portland, Councilor, District 4:5:Number of Winners 3:Joseph (Joe) Alfone:NON","Choice_94

### Note that this is all the voters in the city, not just in D4.

In [4]:
list(D4_df.columns)

['RowNumber',
 'BoxID',
 'BoxPosition',
 'BallotID',
 'PrecinctID',
 'BallotStyleID',
 'PrecinctStyleName',
 'ScanComputerName',
 'Status',
 'Remade',
 'Choice_88_1:City of Portland, Councilor, District 4:1:Number of Winners 3:Bob Weinstein:NON',
 'Choice_88_1:City of Portland, Councilor, District 4:2:Number of Winners 3:Bob Weinstein:NON',
 'Choice_88_1:City of Portland, Councilor, District 4:3:Number of Winners 3:Bob Weinstein:NON',
 'Choice_88_1:City of Portland, Councilor, District 4:4:Number of Winners 3:Bob Weinstein:NON',
 'Choice_88_1:City of Portland, Councilor, District 4:5:Number of Winners 3:Bob Weinstein:NON',
 'Choice_88_1:City of Portland, Councilor, District 4:6:Number of Winners 3:Bob Weinstein:NON',
 'Choice_89_1:City of Portland, Councilor, District 4:1:Number of Winners 3:Andra Vltavin:NON',
 'Choice_89_1:City of Portland, Councilor, District 4:2:Number of Winners 3:Andra Vltavin:NON',
 'Choice_89_1:City of Portland, Councilor, District 4:3:Number of Winners 3:Andra

### Here are all the candidates that the CVR contains rankings for:

In [5]:
D4_candidates = [
    'Olivia Clark',
    'Mitch Green',
    'Eric Zimmerman',
    'Eli Arnold',
    'Chad Lykins',
    'Sarah Silkie',
    'Bob Weinstein',
    'Lisa Freeman',
    'Tony Morse',
    'Ben Hufford',
    'Kevin Goldsmith',
    'Andra Vltavin',
    'Stanley Penkin',
    'John Toran',
    'Chloe Mason',
    'Bob Callahan',
    'Moses Ross',
    'Ciatta R Thompson',
    'Raquel Coyote',
    'Mike DiNapoli',
    'John J Goldsmith',
    'Chris Henry',
    'Joseph (Joe) Alfone',
    'Uncertified Write In',
    'Michael Trimble',
    'Kelly Doyle',
    'Brandon Farley',
    'Patrick Cashman',
    'Tony Schwartz',
    'Lee Odell',
    'Christopher Regis',
    'Write-in-129',
    'Write-in-130',
    'Write-in-131'
]

### Collect columns where 1st place, etc. votes are tallied so we can look for overvotes/undervotes while converting to a better format.

In [6]:
rank1_cols = [col for col in D4_df.columns if '1:Number' in col]
rank2_cols = [col for col in D4_df.columns if '2:Number' in col]
rank3_cols = [col for col in D4_df.columns if '3:Number' in col]
rank4_cols = [col for col in D4_df.columns if '4:Number' in col]
rank5_cols = [col for col in D4_df.columns if '5:Number' in col]
rank6_cols = [col for col in D4_df.columns if '6:Number' in col]

all_rank_cols = rank1_cols + rank2_cols + rank3_cols + rank4_cols + rank5_cols + rank6_cols

### This dataset has rows for ALL Portland voters; let's trim it down to voters who cast at least one vote.

In [7]:
D4_voters_df = D4_df[D4_df[all_rank_cols].sum(axis=1) > 0].reset_index(drop=True)

In [8]:
D4_voters_df 

,RowNumber,BoxID,BoxPosition,BallotID,PrecinctID,BallotStyleID,PrecinctStyleName,ScanComputerName,Status,Remade,"Choice_88_1:City of Portland, Councilor, District 4:1:Number of Winners 3:Bob Weinstein:NON","Choice_88_1:City of Portland, Councilor, District 4:2:Number of Winners 3:Bob Weinstein:NON","Choice_88_1:City of Portland, Councilor, District 4:3:Number of Winners 3:Bob Weinstein:NON","Choice_88_1:City of Portland, Councilor, District 4:4:Number of Winners 3:Bob Weinstein:NON","Choice_88_1:City of Portland, Councilor, District 4:5:Number of Winners 3:Bob Weinstein:NON","Choice_88_1:City of Portland, Councilor, District 4:6:Number of Winners 3:Bob Weinstein:NON","Choice_89_1:City of Portland, Councilor, District 4:1:Number of Winners 3:Andra Vltavin:NON","Choice_89_1:City of Portland, Councilor, District 4:2:Number of Winners 3:Andra Vltavin:NON","Choice_89_1:City of Portland, Councilor, District 4:3:Number of Winners 3:Andra Vltavin:NON","Choice_89_1:City of Portland, Councilor, District 4:4:Number of Winners 3:Andra Vltavin:NON","Choice_89_1:City of Portland, Councilor, District 4:5:Number of Winners 3:Andra Vltavin:NON","Choice_89_1:City of Portland, Councilor, District 4:6:Number of Winners 3:Andra Vltavin:NON","Choice_90_1:City of Portland, Councilor, District 4:1:Number of Winners 3:Lee Odell:NON","Choice_90_1:City of Portland, Councilor, District 4:2:Number of Winners 3:Lee Odell:NON","Choice_90_1:City of Portland, Councilor, District 4:3:Number of Winners 3:Lee Odell:NON","Choice_90_1:City of Portland, Councilor, District 4:4:Number of Winners 3:Lee Odell:NON","Choice_90_1:City of Portland, Councilor, District 4:5:Number of Winners 3:Lee Odell:NON","Choice_90_1:City of Portland, Councilor, District 4:6:Number of Winners 3:Lee Odell:NON","Choice_91_1:City of Portland, Councilor, District 4:1:Number of Winners 3:Kevin Goldsmith:NON","Choice_91_1:City of Portland, Councilor, District 4:2:Number of Winners 3:Kevin Goldsmith:NON","Choice_91_1:City of Portland, Councilor, District 4:3:Number of Winners 3:Kevin Goldsmith:NON","Choice_91_1:City of Portland, Councilor, District 4:4:Number of Winners 3:Kevin Goldsmith:NON","Choice_91_1:City of Portland, Councilor, District 4:5:Number of Winners 3:Kevin Goldsmith:NON","Choice_91_1:City of Portland, Councilor, District 4:6:Number of Winners 3:Kevin Goldsmith:NON","Choice_92_1:City of Portland, Councilor, District 4:1:Number of Winners 3:John J Goldsmith:NON","Choice_92_1:City of Portland, Councilor, District 4:2:Number of Winners 3:John J Goldsmith:NON","Choice_92_1:City of Portland, Councilor, District 4:3:Number of Winners 3:John J Goldsmith:NON","Choice_92_1:City of Portland, Councilor, District 4:4:Number of Winners 3:John J Goldsmith:NON","Choice_92_1:City of Portland, Councilor, District 4:5:Number of Winners 3:John J Goldsmith:NON","Choice_92_1:City of Portland, Councilor, District 4:6:Number of Winners 3:John J Goldsmith:NON","Choice_93_1:City of Portland, Councilor, District 4:1:Number of Winners 3:Mitch Green:NON","Choice_93_1:City of Portland, Councilor, District 4:2:Number of Winners 3:Mitch Green:NON","Choice_93_1:City of Portland, Councilor, District 4:3:Number of Winners 3:Mitch Green:NON","Choice_93_1:City of Portland, Councilor, District 4:4:Number of Winners 3:Mitch Green:NON","Choice_93_1:City of Portland, Councilor, District 4:5:Number of Winners 3:Mitch Green:NON","Choice_93_1:City of Portland, Councilor, District 4:6:Number of Winners 3:Mitch Green:NON","Choice_94_1:City of Portland, Councilor, District 4:1:Number of Winners 3:Joseph (Joe) Alfone:NON","Choice_94_1:City of Portland, Councilor, District 4:2:Number of Winners 3:Joseph (Joe) Alfone:NON","Choice_94_1:City of Portland, Councilor, District 4:3:Number of Winners 3:Joseph (Joe) Alfone:NON","Choice_94_1:City of Portland, Councilor, District 4:4:Number of Winners 3:Joseph (Joe) Alfone:NON","Choice_94_1:City of Portland, Councilor, District 4:5:Number of Winners 3:Joseph (Joe) Alfone:NON","Choice_94

In [9]:
sorted(list(set(D4_voters_df['PrecinctStyleName'])))

['2801-1',
 '2803-1',
 '2804-1',
 '2805-1',
 '2806-1',
 '3301-1',
 '3302-1',
 '3303-1',
 '3304-1',
 '3305-1',
 '3306-1',
 '3307-1',
 '3401-1',
 '3803-1',
 '3804-1',
 '3805-1',
 '3806-1',
 '4101-1',
 '4102-1',
 '4103-1',
 '4105-1',
 '4206-2',
 'CLACK-115-2',
 'CLACK-116-1',
 'WASH-366-1',
 'WASH-375-1',
 'WASH-391-1',
 'WASH-392-1',
 'WASH-441-1']

### So this file DOES include voters in Clackamas and Washington Counties. 

### Add a column for each raw ranking:

In [10]:
D4_voters_df.insert(10, "Rank1 raw", None)
D4_voters_df.insert(11, "Rank2 raw", None)
D4_voters_df.insert(12, "Rank3 raw", None)
D4_voters_df.insert(13, "Rank4 raw", None)
D4_voters_df.insert(14, "Rank5 raw", None)
D4_voters_df.insert(15, "Rank6 raw", None)


### For each voter and each rank, record whether the voter overvoted or undervoted in that rank, or if they voted for exactly one candidate in that rank, record that candidate.

In [11]:
for i in tqdm(D4_voters_df.index):
    
    rank1_votes = D4_voters_df.loc[i, rank1_cols].sum()
    if rank1_votes == 0:
        D4_voters_df.loc[i, 'Rank1 raw'] = "skipped"
    elif rank1_votes > 1:
        D4_voters_df.loc[i, 'Rank1 raw'] = "overvote"
    elif rank1_votes == 1:
        this_vote_col = [col for col in rank1_cols if D4_voters_df.loc[i, col] == 1][0]
        this_vote = [name for name in D4_candidates if name in this_vote_col][0]
        D4_voters_df.loc[i, 'Rank1 raw'] = this_vote

    rank2_votes = D4_voters_df.loc[i, rank2_cols].sum()
    if rank2_votes == 0:
        D4_voters_df.loc[i, 'Rank2 raw'] = "skipped"
    elif rank2_votes > 1:
        D4_voters_df.loc[i, 'Rank2 raw'] = "overvote"
    elif rank2_votes == 1:
        this_vote_col = [col for col in rank2_cols if D4_voters_df.loc[i, col] == 1][0]
        this_vote = [name for name in D4_candidates if name in this_vote_col][0]
        D4_voters_df.loc[i, 'Rank2 raw'] = this_vote

    rank3_votes = D4_voters_df.loc[i, rank3_cols].sum()
    if rank3_votes == 0:
        D4_voters_df.loc[i, 'Rank3 raw'] = "skipped"
    elif rank3_votes > 1:
        D4_voters_df.loc[i, 'Rank3 raw'] = "overvote"
    elif rank3_votes == 1:
        this_vote_col = [col for col in rank3_cols if D4_voters_df.loc[i, col] == 1][0]
        this_vote = [name for name in D4_candidates if name in this_vote_col][0]
        D4_voters_df.loc[i, 'Rank3 raw'] = this_vote

    rank4_votes = D4_voters_df.loc[i, rank4_cols].sum()
    if rank4_votes == 0:
        D4_voters_df.loc[i, 'Rank4 raw'] = "skipped"
    elif rank4_votes > 1:
        D4_voters_df.loc[i, 'Rank4 raw'] = "overvote"
    elif rank4_votes == 1:
        this_vote_col = [col for col in rank4_cols if D4_voters_df.loc[i, col] == 1][0]
        this_vote = [name for name in D4_candidates if name in this_vote_col][0]
        D4_voters_df.loc[i, 'Rank4 raw'] = this_vote

    rank5_votes = D4_voters_df.loc[i, rank5_cols].sum()
    if rank5_votes == 0:
        D4_voters_df.loc[i, 'Rank5 raw'] = "skipped"
    elif rank5_votes > 1:
        D4_voters_df.loc[i, 'Rank5 raw'] = "overvote"
    elif rank5_votes == 1:
        this_vote_col = [col for col in rank5_cols if D4_voters_df.loc[i, col] == 1][0]
        this_vote = [name for name in D4_candidates if name in this_vote_col][0]
        D4_voters_df.loc[i, 'Rank5 raw'] = this_vote

    rank6_votes = D4_voters_df.loc[i, rank6_cols].sum()
    if rank6_votes == 0:
        D4_voters_df.loc[i, 'Rank6 raw'] = "skipped"
    elif rank6_votes > 1:
        D4_voters_df.loc[i, 'Rank6 raw'] = "overvote"
    elif rank6_votes == 1:
        this_vote_col = [col for col in rank6_cols if D4_voters_df.loc[i, col] == 1][0]
        this_vote = [name for name in D4_candidates if name in this_vote_col][0]
        D4_voters_df.loc[i, 'Rank6 raw'] = this_vote


100%|████████████████████████████████████| 77332/77332 [03:03<00:00, 421.41it/s]


In [12]:
D4_voters_df

,RowNumber,BoxID,BoxPosition,BallotID,PrecinctID,BallotStyleID,PrecinctStyleName,ScanComputerName,Status,Remade,Rank1 raw,Rank2 raw,Rank3 raw,Rank4 raw,Rank5 raw,Rank6 raw,"Choice_88_1:City of Portland, Councilor, District 4:1:Number of Winners 3:Bob Weinstein:NON","Choice_88_1:City of Portland, Councilor, District 4:2:Number of Winners 3:Bob Weinstein:NON","Choice_88_1:City of Portland, Councilor, District 4:3:Number of Winners 3:Bob Weinstein:NON","Choice_88_1:City of Portland, Councilor, District 4:4:Number of Winners 3:Bob Weinstein:NON","Choice_88_1:City of Portland, Councilor, District 4:5:Number of Winners 3:Bob Weinstein:NON","Choice_88_1:City of Portland, Councilor, District 4:6:Number of Winners 3:Bob Weinstein:NON","Choice_89_1:City of Portland, Councilor, District 4:1:Number of Winners 3:Andra Vltavin:NON","Choice_89_1:City of Portland, Councilor, District 4:2:Number of Winners 3:Andra Vltavin:NON","Choice_89_1:City of Portland, Councilor, District 4:3:Number of Winners 3:Andra Vltavin:NON","Choice_89_1:City of Portland, Councilor, District 4:4:Number of Winners 3:Andra Vltavin:NON","Choice_89_1:City of Portland, Councilor, District 4:5:Number of Winners 3:Andra Vltavin:NON","Choice_89_1:City of Portland, Councilor, District 4:6:Number of Winners 3:Andra Vltavin:NON","Choice_90_1:City of Portland, Councilor, District 4:1:Number of Winners 3:Lee Odell:NON","Choice_90_1:City of Portland, Councilor, District 4:2:Number of Winners 3:Lee Odell:NON","Choice_90_1:City of Portland, Councilor, District 4:3:Number of Winners 3:Lee Odell:NON","Choice_90_1:City of Portland, Councilor, District 4:4:Number of Winners 3:Lee Odell:NON","Choice_90_1:City of Portland, Councilor, District 4:5:Number of Winners 3:Lee Odell:NON","Choice_90_1:City of Portland, Councilor, District 4:6:Number of Winners 3:Lee Odell:NON","Choice_91_1:City of Portland, Councilor, District 4:1:Number of Winners 3:Kevin Goldsmith:NON","Choice_91_1:City of Portland, Councilor, District 4:2:Number of Winners 3:Kevin Goldsmith:NON","Choice_91_1:City of Portland, Councilor, District 4:3:Number of Winners 3:Kevin Goldsmith:NON","Choice_91_1:City of Portland, Councilor, District 4:4:Number of Winners 3:Kevin Goldsmith:NON","Choice_91_1:City of Portland, Councilor, District 4:5:Number of Winners 3:Kevin Goldsmith:NON","Choice_91_1:City of Portland, Councilor, District 4:6:Number of Winners 3:Kevin Goldsmith:NON","Choice_92_1:City of Portland, Councilor, District 4:1:Number of Winners 3:John J Goldsmith:NON","Choice_92_1:City of Portland, Councilor, District 4:2:Number of Winners 3:John J Goldsmith:NON","Choice_92_1:City of Portland, Councilor, District 4:3:Number of Winners 3:John J Goldsmith:NON","Choice_92_1:City of Portland, Councilor, District 4:4:Number of Winners 3:John J Goldsmith:NON","Choice_92_1:City of Portland, Councilor, District 4:5:Number of Winners 3:John J Goldsmith:NON","Choice_92_1:City of Portland, Councilor, District 4:6:Number of Winners 3:John J Goldsmith:NON","Choice_93_1:City of Portland, Councilor, District 4:1:Number of Winners 3:Mitch Green:NON","Choice_93_1:City of Portland, Councilor, District 4:2:Number of Winners 3:Mitch Green:NON","Choice_93_1:City of Portland, Councilor, District 4:3:Number of Winners 3:Mitch Green:NON","Choice_93_1:City of Portland, Councilor, District 4:4:Number of Winners 3:Mitch Green:NON","Choice_93_1:City of Portland, Councilor, District 4:5:Number of Winners 3:Mitch Green:NON","Choice_93_1:City of Portland, Councilor, District 4:6:Number of Winners 3:Mitch Green:NON","Choice_94_1:City of Portland, Councilor, District 4:1:Number of Winners 3:Joseph (Joe) Alfone:NON","Choice_94_1:City of Portland, Councilor, District 4:2:Number of Winners 3:Joseph (Joe) Alfone:NON","Choice_94_1:City of Portland, Councilor, District 4:3:Number of Winners 3:Joseph (Joe) Alfone:NON","Choice_94_1:City of Portland, Councilor, District 4:4:Number of Winners 3:Joseph (Joe) Alfone:NON","Choice_94_1:City of Portland, Councilor, District

### Next, clean the votes for VoteKit according to Portland's rules: overvotes and undervotes are both skipped and ballots are transferred to the next ranking.
### It appears that the categories 'Write-in-129', 'Write-in-130', and 'Write-in-131' are ALSO skipped and ballots are transferred to the next ranking.  (Equivalently, these "candidates" were eliminated first in a "Round 0" that occurred before the published tabulation rounds.)

In [13]:
D4_voters_df.insert(16, "Rank1 cleaned", None)
D4_voters_df.insert(17, "Rank2 cleaned", None)
D4_voters_df.insert(18, "Rank3 cleaned", None)
D4_voters_df.insert(19, "Rank4 cleaned", None)
D4_voters_df.insert(20, "Rank5 cleaned", None)
D4_voters_df.insert(21, "Rank6 cleaned", None)


### While we do this, save the cleaned ballots to a list to run the election with.

In [14]:
D4_candidates_R1 = D4_candidates[:-3]
D4_candidates_R1

['Olivia Clark',
 'Mitch Green',
 'Eric Zimmerman',
 'Eli Arnold',
 'Chad Lykins',
 'Sarah Silkie',
 'Bob Weinstein',
 'Lisa Freeman',
 'Tony Morse',
 'Ben Hufford',
 'Kevin Goldsmith',
 'Andra Vltavin',
 'Stanley Penkin',
 'John Toran',
 'Chloe Mason',
 'Bob Callahan',
 'Moses Ross',
 'Ciatta R Thompson',
 'Raquel Coyote',
 'Mike DiNapoli',
 'John J Goldsmith',
 'Chris Henry',
 'Joseph (Joe) Alfone',
 'Uncertified Write In',
 'Michael Trimble',
 'Kelly Doyle',
 'Brandon Farley',
 'Patrick Cashman',
 'Tony Schwartz',
 'Lee Odell',
 'Christopher Regis']

In [15]:
cleaned_ballots_R1 = []

for i in tqdm(D4_voters_df.index):
    this_ballot_raw = D4_voters_df.loc[i, ['Rank1 raw', 'Rank2 raw', 'Rank3 raw', 'Rank4 raw', 'Rank5 raw', 'Rank6 raw']]
    
    this_ballot_cleaned = [vote for vote in this_ballot_raw if vote in D4_candidates_R1]

    if len(this_ballot_cleaned) > 0:
        cleaned_ballots_R1.append(this_ballot_cleaned)
    
    if len(this_ballot_cleaned) < 6:
        fill_len = 6 - len(this_ballot_cleaned)
        this_ballot_cleaned = this_ballot_cleaned + ["skipped"]*fill_len

    D4_voters_df.loc[i, "Rank1 cleaned"] = this_ballot_cleaned[0]
    D4_voters_df.loc[i, "Rank2 cleaned"] = this_ballot_cleaned[1]
    D4_voters_df.loc[i, "Rank3 cleaned"] = this_ballot_cleaned[2]
    D4_voters_df.loc[i, "Rank4 cleaned"] = this_ballot_cleaned[3]
    D4_voters_df.loc[i, "Rank5 cleaned"] = this_ballot_cleaned[4]
    D4_voters_df.loc[i, "Rank6 cleaned"] = this_ballot_cleaned[5]
    

100%|███████████████████████████████████| 77332/77332 [00:33<00:00, 2294.15it/s]


### How many first-round votes did each candidate get?

In [16]:
round_1_votes = {}
for name in D4_candidates_R1:
    round_1_votes[name] = 0

for ballot in cleaned_ballots_R1:
    round_1_votes[ballot[0]] += 1

In [17]:
for name in D4_candidates_R1:
    print(name, round_1_votes[name])


Olivia Clark 19138
Mitch Green 10387
Eric Zimmerman 8005
Eli Arnold 8123
Chad Lykins 4757
Sarah Silkie 4415
Bob Weinstein 3891
Lisa Freeman 3749
Tony Morse 2004
Ben Hufford 1694
Kevin Goldsmith 1432
Andra Vltavin 1355
Stanley Penkin 1092
John Toran 1052
Chloe Mason 979
Bob Callahan 888
Moses Ross 578
Ciatta R Thompson 531
Raquel Coyote 317
Mike DiNapoli 315
John J Goldsmith 313
Chris Henry 301
Joseph (Joe) Alfone 257
Uncertified Write In 232
Michael Trimble 230
Kelly Doyle 190
Brandon Farley 168
Patrick Cashman 101
Tony Schwartz 100
Lee Odell 100
Christopher Regis 23


### Excellent - we got the algorithm right!

### How many voters voted in the first round?

In [18]:
len(cleaned_ballots_R1)

76717

### Just for kicks, check the round 2 tallies too.

In [19]:
D4_candidates_R2 = D4_candidates_R1[:-1]

In [20]:
D4_candidates_R2

['Olivia Clark',
 'Mitch Green',
 'Eric Zimmerman',
 'Eli Arnold',
 'Chad Lykins',
 'Sarah Silkie',
 'Bob Weinstein',
 'Lisa Freeman',
 'Tony Morse',
 'Ben Hufford',
 'Kevin Goldsmith',
 'Andra Vltavin',
 'Stanley Penkin',
 'John Toran',
 'Chloe Mason',
 'Bob Callahan',
 'Moses Ross',
 'Ciatta R Thompson',
 'Raquel Coyote',
 'Mike DiNapoli',
 'John J Goldsmith',
 'Chris Henry',
 'Joseph (Joe) Alfone',
 'Uncertified Write In',
 'Michael Trimble',
 'Kelly Doyle',
 'Brandon Farley',
 'Patrick Cashman',
 'Tony Schwartz',
 'Lee Odell']

In [21]:
cleaned_ballots_R2 = []

for ballot in tqdm(cleaned_ballots_R1):
    new_ballot = [name for name in ballot if name in D4_candidates_R2]
    if len(new_ballot) > 0:
        cleaned_ballots_R2.append(new_ballot)

100%|█████████████████████████████████| 76717/76717 [00:00<00:00, 871566.26it/s]


### Round 2 votes:

In [22]:
round_2_votes = {}
for name in D4_candidates_R2:
    round_2_votes[name] = 0

for ballot in cleaned_ballots_R2:
    round_2_votes[ballot[0]] += 1

In [23]:
for name in D4_candidates_R2:
    print(name, round_2_votes[name])


Olivia Clark 19140
Mitch Green 10388
Eric Zimmerman 8005
Eli Arnold 8123
Chad Lykins 4757
Sarah Silkie 4417
Bob Weinstein 3892
Lisa Freeman 3749
Tony Morse 2004
Ben Hufford 1694
Kevin Goldsmith 1432
Andra Vltavin 1355
Stanley Penkin 1092
John Toran 1053
Chloe Mason 980
Bob Callahan 890
Moses Ross 582
Ciatta R Thompson 531
Raquel Coyote 319
Mike DiNapoli 315
John J Goldsmith 313
Chris Henry 302
Joseph (Joe) Alfone 257
Uncertified Write In 232
Michael Trimble 230
Kelly Doyle 190
Brandon Farley 168
Patrick Cashman 102
Tony Schwartz 100
Lee Odell 101


### These are correct too!

### So, let's create two datasets to save: one with the raw data and one with the cleaned data.  Keep all the intro columns for the raw data, but not for the cleaned data.

In [24]:
D4_voters_df

,RowNumber,BoxID,BoxPosition,BallotID,PrecinctID,BallotStyleID,PrecinctStyleName,ScanComputerName,Status,Remade,Rank1 raw,Rank2 raw,Rank3 raw,Rank4 raw,Rank5 raw,Rank6 raw,Rank1 cleaned,Rank2 cleaned,Rank3 cleaned,Rank4 cleaned,Rank5 cleaned,Rank6 cleaned,"Choice_88_1:City of Portland, Councilor, District 4:1:Number of Winners 3:Bob Weinstein:NON","Choice_88_1:City of Portland, Councilor, District 4:2:Number of Winners 3:Bob Weinstein:NON","Choice_88_1:City of Portland, Councilor, District 4:3:Number of Winners 3:Bob Weinstein:NON","Choice_88_1:City of Portland, Councilor, District 4:4:Number of Winners 3:Bob Weinstein:NON","Choice_88_1:City of Portland, Councilor, District 4:5:Number of Winners 3:Bob Weinstein:NON","Choice_88_1:City of Portland, Councilor, District 4:6:Number of Winners 3:Bob Weinstein:NON","Choice_89_1:City of Portland, Councilor, District 4:1:Number of Winners 3:Andra Vltavin:NON","Choice_89_1:City of Portland, Councilor, District 4:2:Number of Winners 3:Andra Vltavin:NON","Choice_89_1:City of Portland, Councilor, District 4:3:Number of Winners 3:Andra Vltavin:NON","Choice_89_1:City of Portland, Councilor, District 4:4:Number of Winners 3:Andra Vltavin:NON","Choice_89_1:City of Portland, Councilor, District 4:5:Number of Winners 3:Andra Vltavin:NON","Choice_89_1:City of Portland, Councilor, District 4:6:Number of Winners 3:Andra Vltavin:NON","Choice_90_1:City of Portland, Councilor, District 4:1:Number of Winners 3:Lee Odell:NON","Choice_90_1:City of Portland, Councilor, District 4:2:Number of Winners 3:Lee Odell:NON","Choice_90_1:City of Portland, Councilor, District 4:3:Number of Winners 3:Lee Odell:NON","Choice_90_1:City of Portland, Councilor, District 4:4:Number of Winners 3:Lee Odell:NON","Choice_90_1:City of Portland, Councilor, District 4:5:Number of Winners 3:Lee Odell:NON","Choice_90_1:City of Portland, Councilor, District 4:6:Number of Winners 3:Lee Odell:NON","Choice_91_1:City of Portland, Councilor, District 4:1:Number of Winners 3:Kevin Goldsmith:NON","Choice_91_1:City of Portland, Councilor, District 4:2:Number of Winners 3:Kevin Goldsmith:NON","Choice_91_1:City of Portland, Councilor, District 4:3:Number of Winners 3:Kevin Goldsmith:NON","Choice_91_1:City of Portland, Councilor, District 4:4:Number of Winners 3:Kevin Goldsmith:NON","Choice_91_1:City of Portland, Councilor, District 4:5:Number of Winners 3:Kevin Goldsmith:NON","Choice_91_1:City of Portland, Councilor, District 4:6:Number of Winners 3:Kevin Goldsmith:NON","Choice_92_1:City of Portland, Councilor, District 4:1:Number of Winners 3:John J Goldsmith:NON","Choice_92_1:City of Portland, Councilor, District 4:2:Number of Winners 3:John J Goldsmith:NON","Choice_92_1:City of Portland, Councilor, District 4:3:Number of Winners 3:John J Goldsmith:NON","Choice_92_1:City of Portland, Councilor, District 4:4:Number of Winners 3:John J Goldsmith:NON","Choice_92_1:City of Portland, Councilor, District 4:5:Number of Winners 3:John J Goldsmith:NON","Choice_92_1:City of Portland, Councilor, District 4:6:Number of Winners 3:John J Goldsmith:NON","Choice_93_1:City of Portland, Councilor, District 4:1:Number of Winners 3:Mitch Green:NON","Choice_93_1:City of Portland, Councilor, District 4:2:Number of Winners 3:Mitch Green:NON","Choice_93_1:City of Portland, Councilor, District 4:3:Number of Winners 3:Mitch Green:NON","Choice_93_1:City of Portland, Councilor, District 4:4:Number of Winners 3:Mitch Green:NON","Choice_93_1:City of Portland, Councilor, District 4:5:Number of Winners 3:Mitch Green:NON","Choice_93_1:City of Portland, Councilor, District 4:6:Number of Winners 3:Mitch Green:NON","Choice_94_1:City of Portland, Councilor, District 4:1:Number of Winners 3:Joseph (Joe) Alfone:NON","Choice_94_1:City of Portland, Councilor, District 4:2:Number of Winners 3:Joseph (Joe) Alfone:NON","Choice_94_1:City of Portland, Councilor, District 4:3:Number of Winners 3:Joseph (Joe) Alfone:NON","Choice_94_1:City of Portland, Councilor, District 4:4:Number of W

In [25]:
D4_voters_df.columns[0:16]

Index(['RowNumber', 'BoxID', 'BoxPosition', 'BallotID', 'PrecinctID',
       'BallotStyleID', 'PrecinctStyleName', 'ScanComputerName', 'Status',
       'Remade', 'Rank1 raw', 'Rank2 raw', 'Rank3 raw', 'Rank4 raw',
       'Rank5 raw', 'Rank6 raw'],
      dtype='object')

In [26]:
D4_raw_df = D4_voters_df[D4_voters_df.columns[0:16]]

In [27]:
D4_voters_df.columns[16:22]

Index(['Rank1 cleaned', 'Rank2 cleaned', 'Rank3 cleaned', 'Rank4 cleaned',
       'Rank5 cleaned', 'Rank6 cleaned'],
      dtype='object')

In [28]:
D4_cleaned_df = D4_voters_df[D4_voters_df.columns[16:22]]

### The raw file is good to go; I just want to rename the ranking columns.

In [29]:
D4_raw_df = D4_raw_df.rename(columns = {
    'Rank1 raw' : 'rank1',
    'Rank2 raw' : 'rank2',
    'Rank3 raw' : 'rank3',
    'Rank4 raw' : 'rank4',
    'Rank5 raw' : 'rank5',
    'Rank6 raw' : 'rank6'
})

In [30]:
D4_raw_df

,RowNumber,BoxID,BoxPosition,BallotID,PrecinctID,BallotStyleID,PrecinctStyleName,ScanComputerName,Status,Remade,rank1,rank2,rank3,rank4,rank5,rank6
0,2,RCV-0001,2,RCV-0001+10005,32,1,2804-1,ScanStation6,0,0,Stanley Penkin,Olivia Clark,skipped,skipped,skipped,skipped
1,3,RCV-0001,3,RCV-0001+10007,53,1,3303-1,ScanStation6,0,0,Chad Lykins,Sarah Silkie,Lisa Freeman,Eric Zimmerman,Olivia Clark,Tony Morse
2,4,RCV-0001,4,RCV-0001+10009,22,1,4105-1,ScanStation6,0,0,Olivia Clark,Sarah Silkie,John Toran,Mitch Green,Lisa Freeman,Bob Callahan
3,6,RCV-0001,6,RCV-0001+10013,66,1,4103-1,ScanStation6,0,0,Mitch Green,Sarah Silkie,Chad Lykins,skipped,skipped,skipped
4,8,RCV-0001,8,RCV-0001+10017,37,1,2806-1,ScanStation6,0,0,Moses Ross,Sarah Silkie,Bob Weinstein,Eric Zimmerman,Olivia Clark,Lisa Freeman
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77327,332947,RCV-1944,8,RCV-1944+10017,34,1,3805-1,ScanStation6,0,0,Mitch Green,Sarah Silkie,skipped,skipped,skipped,skipped
77328,332949,RCV-1944,10,RCV-1944+10021,53,1,3303-1,ScanStation6,0,0,Mitch Green,Chris Henry,Andra Vltavin,Chad Lykins,Sarah Silkie,skipped
77329,332952,RCV-1944,13,RCV-1944+10027,38,1,3307-1,ScanStation6,0,0,Mitch Green,Andra Vltavin,Chad Lykins,Sarah Silkie,Chris Henry,skipped
77330,332955,RCV-1944,16,RCV-1944+10033,59,1,4206-2,ScanStation6,0,0,Lisa Freeman,Chloe Mason,skipped,skipped,skipped,skipped


In [31]:
D4_raw_df.to_csv("./Data/Cast Vote Records/Cleaned CVRs/Portland_D4_Raw.csv", index = False)

### For the cleaned file, I'd like to remove voters who no longer have any votes. (These are the ones who only voted for 'Write-in-129', 'Write-in-130', and/or 'Write-in-131'.)  
### Check that this produces the correct number of first-round votes:

In [32]:
len(D4_cleaned_df[D4_cleaned_df['Rank1 cleaned'] != 'skipped'])

76717

In [33]:
D4_cleaned_df = D4_cleaned_df[D4_cleaned_df['Rank1 cleaned'] != 'skipped'].reset_index(drop="True")

### Rename the columns:

In [34]:
D4_cleaned_df = D4_cleaned_df.rename(columns = {
    'Rank1 cleaned' : 'rank1',
    'Rank2 cleaned' : 'rank2',
    'Rank3 cleaned' : 'rank3',
    'Rank4 cleaned' : 'rank4',
    'Rank5 cleaned' : 'rank5',
    'Rank6 cleaned' : 'rank6'
})

In [35]:
D4_cleaned_df

,rank1,rank2,rank3,rank4,rank5,rank6
0,Stanley Penkin,Olivia Clark,skipped,skipped,skipped,skipped
1,Chad Lykins,Sarah Silkie,Lisa Freeman,Eric Zimmerman,Olivia Clark,Tony Morse
2,Olivia Clark,Sarah Silkie,John Toran,Mitch Green,Lisa Freeman,Bob Callahan
3,Mitch Green,Sarah Silkie,Chad Lykins,skipped,skipped,skipped
4,Moses Ross,Sarah Silkie,Bob Weinstein,Eric Zimmerman,Olivia Clark,Lisa Freeman
...,...,...,...,...,...,...
76712,Mitch Green,Sarah Silkie,skipped,skipped,skipped,skipped
76713,Mitch Green,Chris Henry,Andra Vltavin,Chad Lykins,Sarah Silkie,skipped
76714,Mitch Green,Andra Vltavin,Chad Lykins,Sarah Silkie,Chris Henry,skipped
76715,Lisa Freeman,Chloe Mason,skipped,skipped,skipped,skipped


In [36]:
D4_cleaned_df.to_csv("./Data/Cast Vote Records/Cleaned CVRs/Portland_D4_Cleaned.csv", index = False)